# Cálculo de Betas de los Activos

## Objetivo: Cuantificar la sensibilidad de cada activo respecto al mercado

Este notebook calcula la beta de cada uno de los 60 activos. La beta mide qué tan sensible es un activo a los movimientos del índice de mercado.

**Interpretación de Beta:**
- **β > 1**: Activo agresivo (más volátil que el mercado)
- **β = 1**: Activo neutro (volatilidad igual al mercado)
- **β < 1**: Activo defensivo (menos volátil que el mercado)
- **β < 0**: Activo con movimiento inverso al mercado

**Fórmula:** R_activo = α + β × R_mercado + ε

In [ ]:
# Importaciones
import sys
sys.path.insert(0, '../codigo')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from calcular_betas import CalculadorBetas
import warnings

warnings.filterwarnings('ignore')

# Configuración de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Librerías importadas correctamente")

## Paso 1: Cargar Datos del Excel

Cargamos las 3 hojas: retornos de activos, características, e índice de mercado

In [ ]:
# Inicializar calculador de betas
calculador = CalculadorBetas('../../data/prod_long_sharpe_u60_20260125_v1_train_dataset.csv')

print("\n✓ Datos cargados exitosamente")

## Paso 2: Calcular Betas mediante Regresión Lineal

Para cada activo, calculamos: R_activo = α + β × R_mercado + ε

In [ ]:
# Calcular betas
betas_df = calculador.calcular_betas()

# Mostrar estadísticas resumidas
print("\n" + "="*80)
print("ESTADÍSTICAS RESUMIDAS DE BETAS")
print("="*80)
print(f"\nBeta promedio: {betas_df['Beta'].mean():.4f}")
print(f"Beta mediana: {betas_df['Beta'].median():.4f}")
print(f"Beta mínima: {betas_df['Beta'].min():.4f}")
print(f"Beta máxima: {betas_df['Beta'].max():.4f}")
print(f"Desv. Est. de Beta: {betas_df['Beta'].std():.4f}")

print(f"\nR² promedio (ajuste): {betas_df['R_Squared'].mean():.4f}")
print(f"Correlación promedio: {betas_df['Correlacion'].mean():.4f}")

print("\n" + "="*80)

## Paso 3: Clasificar Activos por Riesgo

In [ ]:
# Clasificar activos por beta
clasificacion = calculador.clasificar_activos()

## Paso 4: Visualizar Betas

In [ ]:
# Visualizar betas
calculador.visualizar_betas(figsize=(16, 12))

## Paso 5: Exportar Resultados y Análisis por Sector

In [ ]:
## Paso 6: Resumen Ejecutivo del Análisis de Betas

In [ ]:
print("\n" + "="*80)
print("RESUMEN EJECUTIVO: ANALISIS DE BETAS")
print("="*80)

print("\n1. ESTADISTICAS GLOBALES:")
print(f"   Beta promedio: {betas_df['Beta'].mean():.4f}")
print(f"   Beta mediana: {betas_df['Beta'].median():.4f}")
print(f"   Rango: [{betas_df['Beta'].min():.4f}, {betas_df['Beta'].max():.4f}]")
print(f"   Desv. Est.: {betas_df['Beta'].std():.4f}")

print("\n2. DISTRIBUCION POR CATEGORIA:")
defensivos_count = len(betas_df[betas_df['Beta'] < 0.8])
neutral_count = len(betas_df[(betas_df['Beta'] >= 0.8) & (betas_df['Beta'] <= 1.2)])
agresivos_count = len(betas_df[betas_df['Beta'] > 1.2])
negativos_count = len(betas_df[betas_df['Beta'] < 0])

print(f"   Defensivos (beta < 0.8): {defensivos_count} activos ({defensivos_count/60*100:.1f}%)")
print(f"   Neutrales (0.8 <= beta <= 1.2): {neutral_count} activos ({neutral_count/60*100:.1f}%)")
print(f"   Agresivos (beta > 1.2): {agresivos_count} activos ({agresivos_count/60*100:.1f}%)")
print(f"   Negativos (beta < 0): {negativos_count} activos ({negativos_count/60*100:.1f}%)")

print("\n3. CALIDAD DEL AJUSTE (R2):")
print(f"   R2 promedio: {betas_df['R_Squared'].mean():.4f}")
print(f"   R2 mediana: {betas_df['R_Squared'].median():.4f}")
print(f"   % con R2 > 0.3: {len(betas_df[betas_df['R_Squared'] > 0.3])/60*100:.1f}%")

print("\n4. ACTIVOS MAS AGRESIVOS (mayor beta):")
top_agresivos = betas_df.nlargest(5, 'Beta')[['Activo', 'Beta', 'Volatilidad_Activo', 'Sector']]
for _, row in top_agresivos.iterrows():
    print(f"   Activo {int(row['Activo']):2}: beta = {row['Beta']:.4f} ({row['Sector'][:30]})")

print("\n5. ACTIVOS MAS DEFENSIVOS (menor beta):")
top_defensivos = betas_df.nsmallest(5, 'Beta')[['Activo', 'Beta', 'Volatilidad_Activo', 'Sector']]
for _, row in top_defensivos.iterrows():
    print(f"   Activo {int(row['Activo']):2}: beta = {row['Beta']:.4f} ({row['Sector'][:30]})")

print("\n6. SECTORES CON MAYOR EXPOSICION AL MERCADO:")
sectors = betas_df.groupby('Sector')['Beta'].agg(['mean', 'count']).sort_values('mean', ascending=False)
for sector, row in sectors.head(5).iterrows():
    print(f"   {sector[:40]:40} | Beta promedio: {row['mean']:.4f} ({int(row['count'])} activos)")

print("\n7. RECOMENDACIONES PARA CONSTRUIR CARTERAS:")
print(f"   [DEFENSIVA] Usar activos con beta < 0.8 ({defensivos_count} disponibles)")
print(f"   [NEUTRAL] Usar activos con 0.8 <= beta <= 1.2 ({neutral_count} disponibles)")
print(f"   [AGRESIVA] Usar activos con beta > 1.2 ({agresivos_count} disponibles)")
print(f"   [HEDGING] Usar Activo 39 con beta negativo para reducir riesgo")

print("\n" + "="*80)

In [ ]:
# Exportar resultados
calculador.exportar_resultados('../betas_resultados.csv')

# Análisis por sector
print("\nAnálisis por sector:")
betas_por_sector = calculador.obtener_betas_por_sector()

# Mostrar tabla final de betas
print("\n" + "="*80)
print("TABLA COMPLETA DE BETAS (primeros 20 activos)")
print("="*80)
print(betas_df[['Activo', 'Beta', 'Alfa', 'R_Squared', 'Correlacion', 'Sector']].head(20).to_string(index=False))